In [36]:
# Librerías
import streamlit as st 
import plotly.express as px
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import statsmodels.api as sm 
from statsmodels.formula.api import ols
from scipy import stats

In [37]:
# Ajustar límites
pd.options.display.max_columns = 90

In [38]:
# Revisar DF
df = pd.read_csv('Berlin_86.csv')
berlin = df.drop(['Unnamed: 0','latitude', 'longitude'], axis=1)
berlin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14187 entries, 0 to 14186
Data columns (total 84 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   listing_url                                   14187 non-null  object 
 1   last_scraped                                  14187 non-null  object 
 2   source                                        14187 non-null  object 
 3   name                                          14187 non-null  object 
 4   description                                   14187 non-null  object 
 5   neighborhood_overview                         14187 non-null  object 
 6   picture_url                                   14187 non-null  object 
 7   host_id                                       14187 non-null  int64  
 8   host_url                                      14187 non-null  object 
 9   host_name                                     14187 non-null 

In [39]:
berlin['host_verifications'].value_counts()

host_verifications
['email', 'phone']                  11337
['email', 'phone', 'work_email']     1785
['phone']                             966
['phone', 'work_email']                58
['email']                              31
[]                                      7
['email', 'work_email']                 3
Name: count, dtype: int64

In [40]:
# Ver DF
berlin

,listing_url,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,property_type,room_type,bathrooms_text,amenities,has_availability,calendar_last_scraped,first_review,last_review,license,instant_bookable,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,availability_eoy,number_of_reviews_ly,estimated_occupancy_l365d,estimated_revenue_l365d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,host_is_superhost_num,host_has_profile_pic_num,host_identity_verified_num,has_availability_num,instant_bookable_num,host_response_rate_cat,review_scores_rating_cat,availability_365_cat,price_cat,beds_cat
0,https://www.airbnb.com/rooms/3176,2025-06-21,city scrape,Fabulous Flat in great Location,This beautiful first floor apartment is situa...,prenzlauer berg,https://a0.muscache.com/pictures/airflow/Hosti...,3718,https://www.airbnb.com/users/show/3718,Britta,2008-10-19,"Coledale, Australia",We love to travel ourselves a lot and prefer t...,within a few hours,f,https://a0.muscache.com/im/users/3718/profile_...,https://a0.muscache.com/im/users/3718/profile_...,prenzlauer berg,"['email', 'phone']",t,t,"Berlin, Germany",prenzlauer berg südwest,Pankow,Entire rental unit,Entire home/apt,1 bath,"[""Iron"", ""Cleaning products"", ""Wifi"", ""Oven"", ...",t,2025-06-21,2009-06-20,2025-03-02,No License,f,100.000000,80.000000,1.0,1.0,2.0,1.0,1.0,2.0,105.000000,63.0,730.0,63.0,63.0,730.0,730.0,63.0,730.0,0,4,4,250,149.0,1.0,0.0,79,0.0,126,13230.0,4.63,4.67,4.52,4.65,4.70,4.92,4.61,1.0,1.0,0.0,0.0,0.76,0,1,1,1,0,Good response,Bad reviews,Great availability,Low price,Few beds
1,https://www.airbnb.com/rooms/9991,2025-06-21,city scrape,Geourgeous flat - outstanding views,4 bedroom with very large windows and outstand...,prenzlauer berg,https://a0.muscache.com/pictures/42799131/59c8...,33852,https://www.airbnb.com/users/show/33852,Philipp,2009-08-25,"Berlin, Germany",Born in Munich - lives in and loves Berlin. Wo...,within a day,f,https://a0.muscache.com/im/users/33852/profile...,https://a0.muscache.com/im/users/33852/profile...,prenzlauer berg,"['email', 'phone']",t,t,"Berlin, Germany",prenzlauer berg südwest,Pankow,Entire rental unit,Entire home/apt,2.5 baths,"[""Iron"", ""Wifi"", ""Babysitter recommendations"",...",t,2025-06-21,2015-08-09,2020-01-04,03/Z/RA/003410-18,f,50.000000,87.618712,1.0,1.0,7.0,2.5,1.0,4.0,135.000000,6.0,14.0,6.0,6.0,14.0,14.0,6.0,14.0,0,0,5,193,7.0,0.0,0.0,29,0.0,0,0.0,5.00,5.00,5.00,5.00,5.00,4.86,4.86,1.0,1.0,0.0,0.0,0.06,0,1,1,1,0,Bad response,Good reviews,Great availability,Low price,Many beds
2,https://www.airbnb.com/rooms/14325,2025-06-21,city scrape,Studio Apartment in Prenzlauer Berg,The apartment is located on the upper second f...,prenzlauer berg,https://a0.muscache.com/pictures/508703/24988a...,55531,https://www.airbnb.com/users/show/55531,Chris + Oliver,2009-11-18,"Berlin, Germany","From Berlin, Germany. We love to travel!",within a day,t,https://a0.muscache.com/im/users/55531/profile...,https://a0.muscache.com/im/users/55531/profile...,prenzlauer berg,"['email', 'phone'

In [41]:
# Chequeo de Null
berlin.isnull().sum()

listing_url                 0
last_scraped                0
source                      0
name                        0
description                 0
                           ..
host_response_rate_cat      0
review_scores_rating_cat    0
availability_365_cat        0
price_cat                   0
beds_cat                    0
Length: 84, dtype: int64

In [42]:
# Ajustar tipos de variables
berlin['host_id'] = berlin['host_id'].astype(str)

In [ ]:
%%writefile Berlin.py
# Se crea el archivo de la APP en el interprete principal (Phyton)
##########
# Importar librerías
import streamlit as st 
import plotly.express as px
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import statsmodels.api as sm 
from statsmodels.formula.api import ols
from scipy import stats

# Definir instancia
@st.cache_resource
#####

# Crear función de carga de datos
def load_data():
    berlin = pd.read_csv('Berlin_86.csv')

    # Ajuste de variables
    df = berlin.drop(['Unnamed: 0','latitude', 'longitude'], axis=1)
    df['host_id'] = df['host_id'].astype(str)

    # Lista
    Lista =['host_is_superhost','host_identity_verified','host_response_time','host_response_rate','host_acceptance_rate','host_total_listings_count','host_verifications','room_type','property_type','price_cat']
    return df, Lista
####
# Carga de datos función 'load_data()'
df, Lista = load_data()

####
# Creación del dashboard
# Generar las páginas a utiizar en el diseño
####
# Generamos encabezados para barra lateral (sidebar)
st.sidebar.title('Berlín, Alemania')

# Widget 1: Selectbox
# Menú desplegable de opciones de las páginas seleccionadas
View = st.sidebar.selectbox(label= 'Tipo de análisis', options= ['Extracción de Características', 'Regresión Lineal', 'Regresión No Lineal', 'Regresión Logística'])

# CONTENIDO DE LA VISTA 1
if View == "Extracción de Características":

    Variable_Cat = st.sidebar.selectbox(label="Variable categórica a analizar", options=Lista)
    Tabla_frecuencias = df[Variable_Cat].value_counts().reset_index().head(10)
    Tabla_frecuencias.columns = ['categorias', 'frecuencia']

    st.title("Extracción de Características — Airbnb Berlín")
    #st.write(f"**Variable seleccionada:** {Variable_Cat}")
    st.caption('Se muestran máximo las 10 categorías con mas frecuencia')

    # FILA 1 — Barras Y Pastel
    Contenedor_A, Contenedor_B = st.columns(2)

    with Contenedor_A:
        st.subheader("Distribución por categoría (Bar Plot)")
        fig_bar = px.bar(
            Tabla_frecuencias,
            x='categorias',
            y='frecuencia',
            color='frecuencia',
            color_continuous_scale='Viridis',
            title=f"Frecuencia por categoría — {Variable_Cat}"
        )
        fig_bar.update_layout(height=400)
        st.plotly_chart(fig_bar, use_container_width=True)

    with Contenedor_B:
        st.subheader("Proporción por categoría (Pie Chart)")
        fig_pie = px.pie(
            Tabla_frecuencias,
            names='categorias',
            values='frecuencia',
            color_discrete_sequence=px.colors.sequential.Tealgrn,
            title=f"Distribución porcentual — {Variable_Cat}"
        )
        st.plotly_chart(fig_pie, use_container_width=True)

    # FILA 2 — Anillo Y Área
    Contenedor_C, Contenedor_D = st.columns(2)

    with Contenedor_C:
        st.subheader("Visualización tipo dona")
        fig_donut = px.pie(
            Tabla_frecuencias,
            names='categorias',
            values='frecuencia',
            hole=0.5,
            color_discrete_sequence=px.colors.sequential.Mint,
            title=f"Gráfico de dona — {Variable_Cat}"
        )
        st.plotly_chart(fig_donut, use_container_width=True)

    with Contenedor_D:
        st.subheader("Tendencia acumulada (Área)")
        fig_area = px.area(
            Tabla_frecuencias.sort_values(by='frecuencia', ascending=False),
            x='categorias',
            y='frecuencia',
            title=f"Tendencia de frecuencia — {Variable_Cat}",
            color_discrete_sequence=['#5C8D89']
        )
        st.plotly_chart(fig_area, use_container_width=True)

    # FILA 3: Heatmap o Boxplot
    st.markdown("---")
    st.subheader("Análisis más profundo")

    if Variable_Cat in ['room_type', 'property_type', 'price_cat']:
        # Relacionar con variable numérica (precio)
        st.write("**Relación entre categorías y precio promedio (Boxplot):**")
        fig_box = px.box(
            df,
            x=Variable_Cat,
            y='price',
            color=Variable_Cat,
            title=f"Distribución de precios según {Variable_Cat}",
            color_discrete_sequence=px.colors.sequential.Viridis
        )
        st.plotly_chart(fig_box, use_container_width=True)
    else:
        # Heatmap simple con frecuencia normalizada
        st.write("**Mapa de calor de proporciones (Heatmap):**")
        heat_df = pd.crosstab(index=df[Variable_Cat], columns='count', normalize='columns') * 100
        fig_heat = px.imshow(
            heat_df,
            color_continuous_scale='Viridis',
            title=f"Proporción por categoría — {Variable_Cat}",
            text_auto=".1f"
        )
        st.plotly_chart(fig_heat, use_container_width=True)

    # Tabla de frecuencias
    st.markdown("---")
    st.subheader("Tabla de frecuencias")
    st.dataframe(Tabla_frecuencias.style.background_gradient(cmap='Blues'))
 ############################################################################


  # Contenido Vista 2
if View == "Regresión Lineal":
    st.title("Regresión Lineal")

    # Variables numéricas disponibles
    numeric_df = df.select_dtypes(include=['float', 'int']).copy()
    Lista_num = list(numeric_df.columns)

    # Lineal simple
    st.subheader("Regresión lineal simple")
    colL, colR = st.columns(2)
    with colL:
        Variable_y = st.selectbox("Variable dependiente (Y)", options=Lista_num, key="rl_y")
    with colR:
        Variable_x = st.selectbox("Variable independiente (X)", options=Lista_num, key="rl_x")

    # Ajuste
    X = numeric_df[[Variable_x]].values
    y = numeric_df[Variable_y].values
    model = LinearRegression()
    model.fit(X, y)
    y_pred = model.predict(X)
    
    # Métricas
    r2 = r2_score(y, y_pred)
    coef_Deter_simple = model.score(X= X, y= y)
    coef_Correl_simple = np.sqrt(coef_Deter_simple)

    # Coeficientes
    coef_df_simple = pd.DataFrame({
        "Variable": [Variable_x],
        "Coeficiente": [model.coef_[0]],
        "Intercepto": [model.intercept_],
        "R: ": [coef_Correl_simple],
        "R^2: ": [coef_Deter_simple]
    })

    st.dataframe(coef_df_simple, use_container_width=True)

    # Graf: dispersión + recta y_pred
    fig_scat = px.scatter(numeric_df, x=Variable_x, y=Variable_y, opacity=0.6, title="Dispersión y recta ajustada")
    # Línea predicha ordenando por X
    order_idx = np.argsort(X[:, 0])
    fig_scat.add_trace(go.Scatter(
        x=X[order_idx, 0], y=y_pred[order_idx],
        mode="lines", name="Predicción de Y"
    ))
    st.plotly_chart(fig_scat, use_container_width=True)

    # Residuales
    resid = y - y_pred
    fig_res = px.scatter(x=y_pred, y=resid, labels={"x":"Variable Independiente", "y":"Residual"},
                         title="Residuos vs Predicción (diagnóstico)")
    fig_res.add_hline(y=0, line_dash="dot")
    st.plotly_chart(fig_res, use_container_width=True)

    st.markdown("---")

    # Lineal múltiple
    st.subheader("Correlación lineal múltiple")
    col1, col2 = st.columns([1,2])
    with col1:
        Variable_y_M = st.selectbox("Variable dependiente (Y)", options=Lista_num, key="rlm_y")
    with col2:
        Variables_x_M = st.multiselect("Variables independientes (X)", options= Lista_num, key="rlm_xs")

    if len(Variables_x_M) >= 1:
        X_M = numeric_df[Variables_x_M].values
        y_M = numeric_df[Variable_y_M].values
        Model_M = LinearRegression()
        Model_M.fit(X_M, y_M)
        y_pred_M = Model_M.predict(X_M)

        # Métricas
        #Corroboramos cual es el coeficiente de Determinación de nuestro modelo
        coef_Deter_multiple= Model_M.score(X=X_M, y= y_M)
        #Corroboramos cual es el coeficiente de Correlación de nuestro modelo
        coef_Correl_multiple = np.sqrt(coef_Deter_multiple)
        #r2M = r2_score(y_M, y_pred_M)
        n, p = X_M.shape

        # Coeficientes
        coef_tab = pd.DataFrame({
            "Variable": ["Intercepto"] + Variables_x_M,
            "Coeficiente": [Model_M.intercept_] + list(Model_M.coef_)
        })
        st.dataframe(coef_tab, use_container_width=True)

        met_tab = pd.DataFrame({'R^2': [coef_Deter_multiple], 'R ': [coef_Correl_multiple]})
        st.dataframe(met_tab, use_container_width=True)

"""        # Gráfica: Real vs Predicho #
        fig_pred = go.Figure()

        # Puntos azules = valores reales (Y_real vs Y_real)
        fig_pred.add_trace(go.Scatter(x= y_M, y= y_M, mode= 'markers', name= 'Y real vs Y real', marker=dict(color="blue", size=6, opacity=0.6)))

        # Puntos rojos = valores predichos (Y_real vs Y_pred)
        fig_pred.add_trace(go.Scatter( x=y_M, y=y_pred_M, mode="markers", name="Y pred vs Y pred", marker=dict(color="red", size=6, opacity=0.6, symbol="circle-open")))

        # Línea ideal y = x's (Referencia de una predicción perfecta)
        fig_pred.add_trace(go.Scatter(x=[y_M.min(), y_M.max()], y=[y_M.min(), y_M.max()], mode="lines", name="Línea ideal", line=dict(color="gray", dash="dot")))

        fig_pred.update_layout(title = 'Comparación: Y real vs Y predicciones', xaxis_title = 'Y real', yaxis_title= 'Y predicciones',legend=dict(title="Leyenda", orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),plot_bgcolor="rgba(0,0,0,0)" )

        st.plotly_chart(fig_pred, use_container_width= True)"""

        # Gráfica: Real vs Predicho 
        fig_pred1 = px.scatter(x=y_M, y=y_pred_M, labels={"x":"Y real ", "y": "Y predicciones"}, title="Comparación Y Real vs Y Predicciones") 
        fig_pred1.add_trace(go.Scatter(x=[y_M.min(), y_M.max()], y=[y_M.min(), y_M.max()], mode="lines", name="Línea ideal", line=dict(dash="dot"))) 
        st.plotly_chart(fig_pred1, use_container_width=True)

        # Mensaje
    else:
        st.info("Selecciona al menos 1 variable para el modelo múltiple.")
    
    #Contenido Vista 3




Overwriting Berlin.py
